In [4]:
import glob
import json
import pandas as pd

In [5]:
files = glob.glob("../gov-projects/*")

In [6]:
"we have %s files" % len(files)

'we have 6638 files'

# Extract projects from files

In [9]:
def extract_projects(file):
    with open(file, "r") as fh:
        data = json.load(fh)
    
    projects = []
    for r in data:
        contract = r['contract'][0]
        del r['contract']

        p = dict(**r, **contract)
        projects.append(p)
    return projects

extract_projects(files[0])[:1]

[{'project_id': '60127147879',
  'project_name': 'ประกวดราคาจ้างก่อสร้างก่อสร้างถนนลาดยางผิวจราจร Cape Seal รหัสทางหลวงท้องถิ่น สท.ถ. 25-001 สายจิกเอน หมู่ที่ 8 บ้านวังไผ่สูง ตำบลไกรนอก กว้าง 6 เมตร ยาว 4,000 เมตร องค์การบริหารส่วนตำบลไกรนอก อำเภอกงไกรลาศ จังหวัดสุโขทัย ด้วยวิธีประกวดราคาอิเล็กทรอนิกส์ (e-bidding)',
  'project_type_name': 'จ้างก่อสร้าง',
  'dept_name': 'องค์การบริหารส่วนตำบลไกรนอก',
  'dept_sub_name': 'องค์การบริหารส่วนตำบลไกรนอก',
  'purchase_method_name': 'ประกวดราคาอิเล็กทรอนิกส์ (e-bidding)',
  'announce_date': '3 ม.ค. 61',
  'project_money': '7,546,000',
  'price_build': '8,708,000',
  'sum_price_agree': '7,510,000',
  'budget_year': '2561',
  'transaction_date': '21 ก.พ. 61',
  'province': 'สุโขทัย',
  'district': 'เมืองสุโขทัย',
  'subdistrict': 'ธานี',
  'project_status': 'ระหว่างดำเนินการ',
  'winner_tin': '0643543000157',
  'corporate_name': 'ห้างหุ้นส่วนจำกัด จามิกรวิศวการ',
  'price_agree': '7,510,000',
  'contract_no': '8/2561',
  'contract_date': '21 ก.พ.

In [10]:
all_projects = []
    
for file in files:
    all_projects.extend(extract_projects(file))
    
df_projects = pd.DataFrame(all_projects)

In [11]:
"we have %d projects" % len(df_projects)

'we have 1085388 projects'

In [12]:
def convert_value_to_int(v):
    v = v.replace(",", "").replace("-", "")
    if v:
        return float(v)
    else:
        return 0

df_projects['sum_price_agree'] = df_projects['sum_price_agree'].apply(convert_value_to_int)

In [13]:
for c in ['price_build', 'project_money', 'price_agree']:
    df_projects[c] = df_projects[c].apply(convert_value_to_int)

# Merge with region


In [14]:
df_province_region = pd.read_csv("../data/province-region.csv")

df_merged = df_projects.merge(df_province_region, on="province")

df_merged.to_csv("../data/2019-local-administrative-organization-projects.csv", index=False)